# Exploratory Data Analysis

In [ ]:
import os
import time
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision.utils import make_grid
from torchvision import transforms
from collections import defaultdict
from torchvision.datasets.folder import pil_loader
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io, transform

%matplotlib inline

## Load Data

We have seven categories of musculoskeletal radiographs

In [ ]:
train_df = pd.read_csv('MURA-v1.0/train.csv', names=['Path', 'Label'])
val_df = pd.read_csv('MURA-v1.0/valid.csv', names=['Path', 'Label'])

Let's checkout the shapes of dataframes

In [ ]:
train_df.shape, val_df.shape

We have 37111 radiographs for training and 3225 radiographs for validation set, let's peak into the dataframes

In [ ]:
train_df.head(3)

In [ ]:
val_df.head(3)

So, we have radiograph paths and their correspoinding labels, each radiographs has a label of 0 (normal) or 1 (abnormal)

## Analysis

### Plot some random radiographs from training and validation set

In [ ]:
train_mat = train_df.as_matrix()
val_mat = val_df.as_matrix()

In [ ]:
ix = np.random.randint(0, len(train_mat)) # randomly select a index
img_path = train_mat[ix][0]
plt.imshow(io.imread(img_path), cmap='binary')
cat = img_path.split('/')[2] # get the radiograph category
plt.title('Category: %s & Lable: %d ' %(cat, train_mat[ix][1]))
plt.show()

In [ ]:
ix = np.random.randint(0, len(val_mat))
img_path = val_mat[ix][0]
plt.imshow(io.imread(img_path), cmap='binary')
cat = img_path.split('/')[2]
plt.title('Category: %s & Lable: %d ' %(cat, val_mat[ix][1]))
plt.show()

This can be seen that images vary in resolution and dimension

In [ ]:
# look at the pixel values
io.imread(img_path)[0]

### Data Exploration

In [ ]:
!ls MURA-v1.0/train/

In [ ]:
!ls MURA-v1.0/train/XR_ELBOW/

So, train dataset has seven study types, each study type has studies on patients stored in folders named like patient001, patient002 etc..

#### Patient count per study type

Let's count number of patients in each study type

In [ ]:
data_cat= ['train', 'valid']
study_types = list(os.walk('MURA-v1.0/train/'))[0][1] # study types, same for train and valid sets
patients_count = {}  # to store all patients count for each study type, for train and valid sets
for phase in data_cat:
    patients_count[phase] = {}
    for study_type in study_types:
        patients = list(os.walk('MURA-v1.0/%s/%s' %(phase, study_type)))[0][1] # patient folder names
        patients_count[phase][study_type] = len(patients)

In [ ]:
print(study_types)
print()
print(patients_count)

In [ ]:
###### plot the patient counts per study type data 
fig = plt.figure(figsize=(10, 15))
for i, phase in enumerate(data_cat):
    ax = fig.add_subplot(2, 1, i+1)
    ax.text(6, max(patients_count[phase].values()) - 40 , phase + ' set')
    x_pos = np.arange(len(study_types))
    plt.bar(x_pos, patients_count[phase].values(), alpha=0.5)
    plt.xticks(x_pos, study_types)
    plt.ylabel('Number of patients')
    plt.xlabel("Study type in '%s' set" % phase)
plt.show()

XR_FINGER has got the most number of patients (1867 in train set, 166 in valid set) followed by XR_WRIST

### Study count

Patients might have multiple studies for a given study type, like a patient may have two studies for wrist, independent of each other. <br> Let's have a look at such cases, here study count = number of patients which have same number of studies

In [ ]:
# let's find out number of studies per study_type
study_count = {} # to store study counts for each study type 
for study_type in study_types:
    BASE_DIR = 'MURA-v1.0/train/%s/' % study_type
    study_count[study_type] = defaultdict(lambda:0) # to store study count for current study_type, initialized to 0 by default
    patients = list(os.walk(BASE_DIR))[0][1] # patient folder names
    for patient in patients:
        studies = os.listdir(BASE_DIR+patient)
        study_count[study_type][len(studies)] += 1

In [ ]:
study_count

XR_WRIST has 3111 patients who have only single study, similarly, 158 patients have 2 studies, 12 patients have 3 studies and 4 patients have 4 studies. <br> let's plot this data

In [ ]:
# plot the study count vs number of patients per study type data 
fig = plt.figure(figsize=(10, 30))
for i, study_type in enumerate(study_count):
    ax = fig.add_subplot(7, 1, i+1)
    study = study_count[study_type]
    # text in the plot
    m = max(study.values())
    for i, v in enumerate(study.values()):
        if v==m: ax.text(i, v - 200, str(v))
        else: ax.text(i, v + 200, str(v))
    ax.text(i, m - 100, study_type, color='green')
    # plot the bar chart
    x_pos = np.arange(len(study))
    plt.bar(x_pos, study.values(), align='center', alpha=0.5)
    plt.xticks(x_pos,  study.keys())
    plt.xlabel('Number of studies')
    plt.ylabel('Number of patients')
plt.show()

### Number of views per study

It can be seen that each study may have more that one view (radiograph image), let' have a look

In [ ]:
# let's find out number of studies per study_type
view_count = {} # to store study counts for each study type, study count = number of patients which have similar number of studies 
for study_type in study_types:
    BASE_DIR = 'MURA-v1.0/train/%s/' % study_type
    view_count[study_type] = defaultdict(lambda:0) # to store study count for current study_type, initialized to 0 by default
    patients = list(os.walk(BASE_DIR))[0][1] # patient folder names
    for patient in patients:
        studies = os.listdir(BASE_DIR + patient)
        for study in studies:
            views = os.listdir(BASE_DIR + patient + '/' + study)
            view_count[study_type][len(views)] += 1

In [ ]:
view_count

`XR_SHOULDER` has as many as 13 views in some studies, `XR_HAND` has 5 at max, this poses a challenging task to predict on a study taking into account all the views of that study while keeping the batch size of 8 (as mentioned in MURA paper)

In [ ]:
# plot the view count vs number of studies per study type data 
fig = plt.figure(figsize=(10, 30))
for i, view_type in enumerate(view_count):
    ax = fig.add_subplot(7, 1, i+1)
    view = view_count[view_type]
    # text in the plot
    m = max(view.values())
    for i, v in enumerate(view.values()):
        if v==m: ax.text(i, v - 200, str(v))
        else: ax.text(i, v + 80, str(v))
    ax.text(i - 0.4, m - 80, view_type, color='green')
    # plot the bar chart
    x_pos = np.arange(len(view))
    plt.bar(x_pos, view.values(), align='center', alpha=0.5)
    plt.xticks(x_pos,  view.keys())
    plt.xlabel('Number of views')
    plt.ylabel('Number of studies')
plt.show()

Most of the studies contain 2, 3 or 4 views

## Data Augmentation and Training Data Pipeline

In [ ]:
class ImageDataset(Dataset):
    """training dataset."""

    def __init__(self, df, transform=None):
        """
        Args:
            df (pd.DataFrame): a pandas DataFrame with image path and labels.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = self.df.iloc[idx, 0]
        image = pil_loader(img_name)
        label = self.df.iloc[idx, 1]
        sample = {'image': image, 'label': label}

        if self.transform:
            sample['image'] = self.transform(image)
            
        return sample

In [ ]:
train_df = pd.read_csv('MURA-v1.0/train.csv', names=['Path', 'Label'])
val_df = pd.read_csv('MURA-v1.0/valid.csv', names=['Path', 'Label'])

In [ ]:
data = {
    'train': train_df[train_df['Path'].str.contains("WRIST")],
    'val': val_df[val_df['Path'].str.contains("WRIST")]
}

In [ ]:
data_cat = ['train', 'val'] # data categories
data_transforms = {
    'train': transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(10),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
image_datasets = { x: ImageDataset(data[x], transform=data_transforms[x]) for x in data_cat}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=4) for x in data_cat}

In [ ]:
# plot a batch sample from `wrist_dataloaders`
fig = plt.figure(figsize=(15, 15))

def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
sample = next(iter(dataloaders['train']))

# Make a grid from batch
out = make_grid(sample['image'])

imshow(out, title=[l for l in sample['label']])